This notebook will be parameterized to select a subset of data to write

In [0]:
# Acquire data at /databricks-datasets/retail-org/compnay_employees/complany_employees.csv
# dbutils.fs.head("/databricks-datasets/retail-org/company_employees/company_employees.csv")
emp_df = spark.read.csv("/databricks-datasets/retail-org/company_employees/company_employees.csv", header=True)

In [0]:
display(emp_df)

In [0]:
# Create a parameter called department
dbutils.widgets.text("dept", "", "Enter name of department")

In [0]:
# Print parameter value
_dept = dbutils.widgets.get("dept")

In [0]:
# Use variable to filter data
df_filtered = emp_df.filter((emp_df.department == _dept.upper()) & (emp_df.active_record == 1))

In [0]:
# Write to table only if data exists
_count = df_filtered.count()
if _count > 0:
    print(f"Data exists, writing to {_dept}_table")
    table_name = f"{_dept}_table"
    df_filtered.write.format("delta").mode("overwrite").saveAsTable(f"dev.bronze.{table_name}")
    display(spark.table(f"dev.bronze.{table_name}")) # Reads Delta table
else:
    print(f"No data for {_dept}")

In [0]:
# Exit this notebook wit the current number of records written
dbutils.notebook.exit(_count)